In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

# Load the TFLite model
model_path = "manipulation_detector_config_7.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Preprocesses an image before it is given to the model to ensure consistency
# across all images that the model is learning off of.
def preprocess_image(image_path):
    # Load the image and convert it to RGB colour space; 
    # we want all images to be in same colour space for consistency.
    with Image.open(image_path) as image:
        image = image.convert("RGB")
    image = image.resize((256, 256))
    # Convert the image to an array. Normalize its pixel values to be between 0 and 1.
    # Since most images in the training/test dataset are jpegs and in RGB, assume that
    # we can divide each pixel by 255 to normalize them to be between 0 and 1.
    preprocessed_image = np.array(image) / 255.0
    # Return the preprocessed image.
    return preprocessed_image

# Define a function to classify a single image
def classify_image(image_path):
    input_data = preprocess_image(image_path)
    input_data = np.expand_dims(input_data, axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data[0][0]

# Define the folder where the images are located
folder_path = "reserve_2"

# Initialize counters for classification results
num_authentic = 0
num_manipulated = 0
num_au_authentic = 0
num_au_manipulated = 0
num_tp_authentic = 0
num_tp_manipulated = 0
num_au_filenames = 0
num_tp_filenames = 0

# Iterate over all the images in the folder
for filename in os.listdir(folder_path):
    if (filename.endswith(".jpg") or filename.endswith(".jpeg")):
        # Classify the image and update counters
        image_path = os.path.join(folder_path, filename)
        classification_result = classify_image(image_path)
        if classification_result < 0.5:
            print(f"{filename} - Manipulated: {classification_result}")
            num_manipulated += 1
            if (filename.startswith("Au") or filename.startswith("au")):
                num_au_manipulated += 1
                num_au_filenames += 1
            elif (filename.startswith("Tp") or filename.startswith("tp")):
                num_tp_manipulated += 1
                num_tp_filenames += 1
        else:
            print(f"{filename} - Authentic: {classification_result}")
            num_authentic += 1
            if (filename.startswith("Au") or filename.startswith("au")):
                num_au_authentic += 1
                num_au_filenames += 1
            elif (filename.startswith("Tp") or filename.startswith("tp")):
                num_tp_authentic += 1
                num_tp_filenames += 1

# Print the classification results and filename counts
print(f"Total authentic images: {num_authentic}")
print(f"Total manipulated images: {num_manipulated}")
print(f"Number of 'au' authentic images: {num_au_authentic}")
print(f"Number of 'au' manipulated images: {num_au_manipulated}")
print(f"Number of 'tp' authentic images: {num_tp_authentic}")
print(f"Number of 'tp' manipulated images: {num_tp_manipulated}")
print(f"Number of 'au' filenames: {num_au_filenames}")
print(f"Number of 'tp' filenames: {num_tp_filenames}")

precision = num_au_authentic/(num_au_authentic + num_tp_authentic)
recall = num_au_authentic/(num_au_authentic + num_au_manipulated)
accuracy = (num_au_authentic + num_tp_manipulated)/(num_au_authentic + num_tp_authentic + num_au_manipulated + num_tp_manipulated)

print(f"precision: {precision}")
print(f"recall: {recall}")
print(f"accuracy: {accuracy}")



Au_pla_30535.jpg - Authentic: 0.9995783567428589
Tp_S_NRD_S_N_pla20018_pla20018_02489.jpeg - Authentic: 0.7959454655647278
Tp_S_NNN_S_B_nat00029_nat00029_00645.jpeg - Authentic: 0.8162985444068909
Au_pla_30253.jpg - Manipulated: 0.0013904757797718048
Au_pla_20040.jpg - Manipulated: 0.003384819021448493
Tp_S_NNN_S_N_arc00076_arc00076_01052.jpeg - Manipulated: 0.22925539314746857
Tp_S_NNN_S_N_cha00015_cha00015_00844.jpeg - Authentic: 0.9750930070877075
Au_pla_30247.jpg - Manipulated: 0.0064665828831493855
Au_pla_20054.jpg - Manipulated: 9.432427759747952e-05
Tp_S_NNN_S_N_arc00027_arc00027_01095.jpeg - Authentic: 0.9964579343795776
Au_pla_30521.jpg - Authentic: 0.9842270612716675
Tp_S_NRN_S_N_pla00065_pla00065_01172.jpeg - Authentic: 0.7421718239784241
Tp_S_NNN_S_N_art20027_art20027_01833.jpeg - Manipulated: 0.051797449588775635
Au_pla_30509.jpg - Authentic: 0.9999998211860657
Tp_S_NRN_S_N_ind00055_ind00055_00905.jpeg - Manipulated: 0.16376090049743652
Au_pla_20068.jpg - Manipulated: 0.00

Au_sec_00057.jpg - Authentic: 0.9998788237571716
Au_sec_30194.jpg - Authentic: 0.9990425705909729
Tp_S_NNN_S_N_ind00063_ind00063_00909.jpeg - Manipulated: 0.059695061296224594
Tp_S_NNN_S_N_arc10101_arc10101_11168.jpg - Authentic: 0.9176716804504395
Au_sec_00043.jpg - Authentic: 0.9112704992294312
Au_sec_30180.jpg - Authentic: 0.980092465877533
Tp_S_NNN_S_N_sec20034_sec20034_01624.jpeg - Manipulated: 0.012073112651705742
Tp_S_NNN_S_N_art10010_art10010_20093.jpg - Authentic: 0.9252753853797913
Au_nat_30389.jpg - Authentic: 0.9968410134315491
Au_pla_30469.jpg - Manipulated: 0.45674270391464233
Au_pla_30482.jpg - Manipulated: 0.00011591138900257647
Tp_S_NNN_S_B_nat20002_nat20002_02202.jpeg - Manipulated: 0.04237201064825058
Au_nat_30404.jpg - Authentic: 0.8210775852203369
Tp_S_NNN_S_B_sec20080_sec20080_01670.jpeg - Manipulated: 0.06653746217489243
Au_nat_30362.jpg - Authentic: 0.9940938949584961
Tp_S_NNN_S_N_pla00031_pla00031_10985.jpg - Manipulated: 6.377304089255631e-05
Tp_S_NRN_S_B_nat2

Au_pla_30050.jpg - Authentic: 0.9849119186401367
Tp_S_NNN_S_N_cha00049_cha00049_00369.jpeg - Manipulated: 0.23134976625442505
Tp_S_NNN_S_O_ani00021_ani00021_20026.jpg - Manipulated: 0.009665997698903084
Au_pla_30736.jpg - Authentic: 0.9915149807929993
Tp_S_NNN_S_N_ani00071_ani00071_00827.jpeg - Authentic: 0.9098073840141296
Au_sec_30036.jpg - Authentic: 0.999427318572998
Au_nat_30559.jpg - Authentic: 0.999603271484375
Tp_S_NNN_S_N_cha10181_cha10181_12296.jpg - Authentic: 0.7793054580688477
Tp_S_NNN_S_N_nat20055_nat20055_02250.jpeg - Manipulated: 0.0014963269932195544
Au_nat_20038.jpg - Manipulated: 0.21616743505001068
Au_sec_30022.jpg - Authentic: 0.8250012397766113
Tp_S_NRN_S_N_cha10130_cha10130_12186.jpg - Manipulated: 0.381273478269577
Tp_S_NNN_S_B_ind00056_ind00056_01345.jpeg - Manipulated: 0.04711734876036644
Au_nat_20010.jpg - Manipulated: 0.3897691071033478
Au_nat_30203.jpg - Authentic: 0.9733707904815674
Tp_S_NNN_S_B_nat20001_nat20001_01505.jpeg - Manipulated: 0.056625224649906

Au_pla_30126.jpg - Authentic: 0.5303375720977783
Au_nat_30188.jpg - Authentic: 0.9850820302963257
Au_sec_30381.jpg - Authentic: 0.9998584985733032
Tp_S_NRN_S_N_arc20065_arc20065_01705.jpeg - Manipulated: 0.4705638885498047
Tp_S_NNN_S_N_ani10222_ani10222_12395.jpg - Authentic: 0.9170040488243103
Tp_S_NRN_S_N_cha20042_cha20042_02041.jpeg - Authentic: 0.7441608905792236
Au_pla_30668.jpg - Manipulated: 0.0683765634894371
Au_txt_00015.jpg - Manipulated: 0.286151260137558
Tp_S_NNN_S_N_nat20053_nat20053_01557.jpeg - Manipulated: 0.00677836500108242
Tp_S_NNN_S_N_sec00077_sec00077_00752.jpeg - Manipulated: 0.07311146706342697
Au_txt_00001.jpg - Manipulated: 0.3749470114707947
Au_sec_30395.jpg - Manipulated: 0.33293846249580383
Tp_S_NNN_S_B_sec10001_sec10001_20007.jpg - Manipulated: 0.03861078992486
Tp_S_NNN_S_N_pla00011_pla00011_00550.jpeg - Manipulated: 0.0004490762948989868
Au_nat_30177.jpg - Authentic: 0.9987519979476929
Au_pla_00032.jpg - Manipulated: 0.008714786730706692
Au_sec_30418.jpg -

Tp_S_NNN_S_N_nat20026_nat20026_01530.jpeg - Manipulated: 0.08119586855173111
Tp_S_NNN_S_N_ind20050_ind20050_02296.jpeg - Authentic: 0.9983301162719727
Au_sec_30419.jpg - Manipulated: 0.0017378340708091855
Tp_S_NNN_S_N_ind00029_ind00029_00896.jpeg - Manipulated: 0.021491238847374916
Au_sec_30394.jpg - Manipulated: 0.4826469123363495
Tp_S_NNN_S_N_nat20050_nat20050_02246.jpeg - Manipulated: 0.15227928757667542
Au_sec_30380.jpg - Authentic: 0.9999902844429016
Tp_S_NNN_S_N_nat10107_nat10107_11348.jpg - Manipulated: 0.03946402296423912
Au_nat_30189.jpg - Authentic: 0.9947987198829651
Au_txt_00014.jpg - Manipulated: 0.22294938564300537
Au_pla_30669.jpg - Authentic: 0.9952866435050964
Tp_S_NNN_S_N_ani00089_ani00089_11145.jpg - Authentic: 0.9563140869140625
Au_pla_30641.jpg - Authentic: 0.9447171688079834
Au_pla_30127.jpg - Authentic: 0.8145750761032104
Tp_S_NRN_S_N_ind00058_ind00058_10426.jpeg - Manipulated: 0.002207766752690077
Tp_S_NRN_S_N_art00043_art00043_01411.jpeg - Manipulated: 0.213658

...

Au_sec_30014.jpg - Authentic: 0.9955559968948364
Au_pla_20088.jpg - Manipulated: 0.001778647885657847
Tp_S_NNN_S_B_nat00019_nat00019_20032.jpg - Manipulated: 0.31676971912384033
Au_nat_20026.jpg - Manipulated: 0.0635952278971672
Au_nat_30235.jpg - Manipulated: 0.04176156222820282
Tp_S_NNN_S_B_ind00078_ind00078_01364.jpeg - Manipulated: 0.015525824390351772
Au_nat_30553.jpg - Authentic: 0.999783992767334
Au_nat_30547.jpg - Authentic: 1.0
Au_sec_30028.jpg - Authentic: 0.9999949336051941
Tp_S_NRN_S_O_cha00035_cha00067_11734.jpg - Manipulated: 0.3618088960647583
Au_nat_20032.jpg - Manipulated: 0.002724943682551384
Au_nat_30221.jpg - Authentic: 0.9130231738090515
Tp_S_NNN_S_N_arc20099_arc20099_02186.jpeg - Authentic: 0.7326977849006653
Tp_S_NNN_S_N_arc00024_arc00024_01098.jpeg - Authentic: 0.9754481911659241
Tp_S_NNN_S_N_arc00096_arc00096_01034.jpeg - Authentic: 0.6725860238075256
Au_sec_00100.jpg - Authentic: 0.811142086982727
Tp_S_NNN_S_B_txt00028_txt00028_20117.jpeg - Manipulated: 0.0046

Tp_S_NRD_S_N_cha10154_cha10154_12266.jpg - Authentic: 0.9620179533958435
Au_nat_30621.jpg - Authentic: 0.7847045660018921
Tp_S_NNN_S_N_arc20028_arc20028_02169.jpeg - Authentic: 0.9284380078315735
Tp_S_NRN_S_N_pla20088_pla20088_01988.jpeg - Manipulated: 0.0002857409417629242
Au_pla_30328.jpg - Authentic: 0.9995783567428589
Au_sec_00064.jpg - Authentic: 0.8523094058036804
Au_sec_00070.jpg - Manipulated: 0.028923414647579193
Tp_S_NRD_S_N_ani00078_ani00078_00822.jpeg - Manipulated: 0.02384720742702484
Au_pla_30314.jpg - Manipulated: 0.048106640577316284
Au_sec_00058.jpg - Authentic: 0.9976931810379028
Au_nat_30392.jpg - Authentic: 0.9989649057388306
Tp_S_NRD_S_N_arc20101_arc20101_01741.jpeg - Manipulated: 0.016754677519202232
Tp_S_NNN_S_N_art00099_art00099_11714.jpg - Manipulated: 0.0332336351275444
Au_pla_30472.jpg - Manipulated: 0.002854269463568926
Tp_S_NRN_S_N_sec00070_sec00070_10717.jpg - Manipulated: 0.06919772922992706
Au_pla_30466.jpg - Authentic: 0.9500409364700317
Tp_S_NNN_S_N_an

Au_sec_10114.jpg - Manipulated: 0.3021330237388611
Tp_S_NNN_S_N_ind00078_ind00078_10662.jpg - Manipulated: 0.014475226402282715
Au_pla_30739.jpg - Authentic: 0.9165045619010925
Tp_S_NRN_S_N_ani00033_ani00033_00153.jpeg - Authentic: 0.6824272274971008
Tp_S_NRD_S_B_nat00038_nat00038_00957.jpeg - Manipulated: 0.006340859457850456
Tp_S_NNN_S_N_sec20074_sec20074_01664.jpeg - Authentic: 0.589623212814331
Tp_S_NNN_S_N_sec00036_sec00036_00737.jpeg - Manipulated: 0.05863043665885925
Tp_S_NNN_S_N_ind00026_ind00026_00441.jpeg - Manipulated: 0.047994647175073624
Tp_S_NNN_S_N_cha00058_cha00058_00854.jpeg - Authentic: 0.5040601491928101
Tp_S_NRN_M_N_art00015_art00015_10272.jpeg - Authentic: 0.6623929142951965
Au_nat_30556.jpg - Authentic: 0.9998412728309631
Au_nat_30230.jpg - Authentic: 0.9998095631599426
Au_nat_20023.jpg - Manipulated: 0.004306419752538204
Au_sec_30039.jpg - Authentic: 0.9999948143959045
Au_nat_30224.jpg - Authentic: 0.9747212529182434
Au_nat_20037.jpg - Manipulated: 0.023817123845

Au_sec_30417.jpg - Authentic: 0.9054790735244751
Au_pla_30698.jpg - Authentic: 0.8402867317199707
Tp_S_NRN_S_N_cha00081_cha00081_10190.jpeg - Manipulated: 0.09364684671163559
Au_sec_30371.jpg - Authentic: 0.9968422055244446
Au_nat_30178.jpg - Authentic: 0.738185703754425
Tp_S_NNN_S_B_sec00083_sec00083_11286.jpg - Manipulated: 0.00041591780609451234
Tp_S_NNN_S_N_sec20026_sec20026_02452.jpeg - Manipulated: 0.38093796372413635
Au_pla_00029.jpg - Manipulated: 4.237297980580479e-05
Tp_S_NNN_S_N_sec00056_sec00056_00732.jpeg - Manipulated: 0.32202666997909546
Au_sec_30365.jpg - Authentic: 0.9999983310699463
Tp_S_NNN_S_N_pla20100_pla20100_02342.jpeg - Manipulated: 0.005733469966799021
Au_sec_30403.jpg - Authentic: 0.998120903968811
Au_nat_30622.jpg - Authentic: 0.9827772974967957
Tp_S_NRN_S_N_nat10144_nat10144_11973.jpg - Manipulated: 0.005593350622802973
Tp_S_NRN_S_N_ind20015_ind20015_01756.jpeg - Authentic: 0.810897946357727
Au_nat_30144.jpg - Authentic: 0.9405418038368225
Au_pla_00001.jpg -

Tp_S_NRN_S_N_cha10196_cha10196_12304.jpg - Authentic: 0.9959281086921692
Au_nat_30186.jpg - Authentic: 0.9975172877311707
Au_pla_30100.jpg - Manipulated: 0.016453120857477188
Au_pla_30128.jpg - Manipulated: 0.0062738461419939995
Au_txt_00033.jpg - Manipulated: 0.12500818073749542
Tp_S_NNN_S_N_art20054_art20054_02331.jpeg - Manipulated: 0.12653785943984985
Tp_S_NNN_S_N_cha00041_cha00041_00361.jpeg - Authentic: 0.5820728540420532
Au_txt_00027.jpg - Authentic: 0.6713106036186218
Tp_S_NNN_S_N_ind00029_ind00029_01322.jpeg - Manipulated: 0.01949661783874035
Tp_S_NNN_S_N_arc20071_arc20071_01711.jpeg - Authentic: 0.711005449295044
Au_nat_30025.jpg - Authentic: 0.9997336268424988
Tp_S_NNN_S_B_nat00015_nat00015_11052.jpg - Manipulated: 0.3873017132282257
Tp_S_NRN_S_N_ind20030_ind20030_01771.jpeg - Authentic: 0.9990177154541016
Tp_S_NNN_S_N_ind20019_ind20019_02276.jpeg - Authentic: 0.5260191559791565
Tp_S_NNN_S_B_sec20060_sec20060_02148.jpeg - Authentic: 0.5423409342765808
Au_sec_30238.jpg - Auth